In [56]:
from torch_geometric_temporal.signal import temporal_signal_split
from temporal_gnn_dataset_loader import temporalGNNLoader

loader = temporalGNNLoader("../../Datasets/study_373/top_5_373.csv")
dataset = loader.get_dataset(edges=[[0, 2], [0, 4], [1, 2], [1, 4], [3, 4]])
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.7)


ModuleNotFoundError: No module named 'temporal_gnn_dataset_loader'

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvLSTM


class GTCLSTM(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, K):
        super(GTCLSTM, self).__init__()
        self.lstm = GConvLSTM(input_dim, hidden_dim, K)
        self.linear = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_weight):
        x = self.lstm(x, edge_index, edge_weight)
        x = self.linear(x[1])
        return x

In [ ]:
from tqdm import tqdm

model = GTCLSTM(input_dim=4, hidden_dim=32, output_dim=4, K=2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(50)):
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr).T
        cost = torch.mean((y_hat-snapshot.y)**2)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr).T
    print(y_hat)
    print(snapshot.y)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

In [ ]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [ ]:
print(dataset.features)